# Tugas 1 : Crawling Berita

**Apa itu crawling ?**


Perayapan data, atau data crawling adalah sebuah proses yang menggali jauh ke dalam world wide web atau target tertentu untuk mengambil data. Perayapan data juga didefinisikan sebagai proses otomatis untuk mengumpulkan dan mengindeks data dari berbagai sumber.

Aktivitas ini dilakukan oleh bot atau software yang disebut dengan crawler. Data yang diambil dari hasil crawling ini pada umumnya akan dianalisis, dijadikan bahan pengembangan sistem, atau bahkan dijadikan data penelitian tertentu.

Proses pengambilan data dari proses perayapan ini diawali dengan crawler yang mulai memasuki target yang telah ditentukan. Setelah itu, crawler akan mengambil data-data penting yang dibutuhkan.

Pada umumnya, data-data yang diambil dari proses ini berupa spesifikasi produk, harga, kategori, dan lain-lain.

**Fungsi Data Crawling**

1. Data Statistik

  Fungsi data crawling yang pertama adalah untuk kebutuhan data statistik. Pada dasarnya, proses perayapan data memang berfungsi untuk mengumpulkan data-data tertentu, termasuk informasi statistik.

  Data statistik yang diperoleh ini nantinya akan digunakan sebagai bahan analisis. Data statistik yang pada umumnya diambil melalui proses crawling adalah data analisis pasar, analisis calon konsumen, dan lain-lain.

2. Membandingkan Detail Sebuah Produk

  Ketika Anda mencari sebuah produk di internet, kemudian berniat untuk membandingkan detail produk tersebut seperti spesifikasi, harga, serta kategorinya, di sinilah perayapan data berperan.

  Dengan bantuan perayapan data, Anda bisa menemukan produk di mesin pencari, seiring dengan preferensi lainnya yang terkait dengan produk tersebut.

**Source**

Pada proses crawling ini sumber data diambil dari  https://health.detik.com, Detik Health adalah salah satu sub-kanal dari situs berita Detik.com yang khusus membahas topik kesehatan. Detik Health bertujuan untuk menyediakan informasi yang bermanfaat bagi pembaca yang ingin menjaga kesehatan atau mencari solusi atas masalah kesehatan mereka. Data yang diambil diantaranya :


*   Judul

* Isi berita

* Tanggal

* Kategori Berita

pada tugas crawling berita kali ini hanya mengambil data sampel berita sebanyak 10 artikel


**Proses Crawling**

Dalam proses crawling ini saya menggunakan beberapa library dan diantara library yang penting di import adalah *BeautifulSoup* yang berfungsi sebagai library crawler

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, min_articles_per_category):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len([k for k in kategori_list if k == kategori]) >= min_articles_per_category:
            return  # Menghentikan proses jika jumlah artikel sudah mencapai minimum yang diinginkan

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        judul.append(title)
        tanggal.append(date)
        isi.append(content)
        kategori_list.append(kategori)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://health.detik.com/berita-detikhealth/indeks",
             "https://sport.detik.com/indeks",
]
categories = [
    "Olahraga",
    "Pariwisata",
]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []
kategori_list = []

# Batas minimal artikel per kategori
min_articles_per_category = 50

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < min_articles_per_category:
        url = f"{base_url}/{page}"
        get_data(url, category, min_articles_per_category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman
        page += 1

# Membuat dataframe dari list data
df = pd.DataFrame({"judul": judul, "tanggal": tanggal, "isi": isi, "kategori": kategori_list})

# Menyimpan dataframe ke file csv
df.to_csv("data_berita_detik.csv", index=False)

# Menampilkan dataframe
print(df)


In [ ]:
# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        # Menambahkan data ke dalam list
        judul.append(title)
        tanggal.append(date)
        isi.append(content)
        kategori_list.append(kategori)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://health.detik.com/berita-detikhealth/indeks",
             "https://sport.detik.com/indeks",
]

categories = ["Kesehatan",
              "Olahraga",
]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []
kategori_list = []

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    for page in range(1, 4):  # Looping untuk beralih halaman
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            break

        url = f"{base_url}/{page}"
        get_data(url, category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman

# Membuat dataframe dari list data
df = pd.DataFrame({
    "judul": judul,
    "isi": isi,
    "tanggal": tanggal,
    "kategori": kategori_list
})

# Menyimpan dataframe ke file CSV
df.to_csv("Crawl-berita.csv", index=False)


Apakah Obat Statin Cepat Turunkan Kadar Kolesterol? Dokter Bilang Gini
Sesal IDI soal Viral Dokter Dilarang Pakai Hijab di RS Medistra
5 Cara Menurunkan Kolesterol Secara Alami, Salah Satunya Kurangi Makan Ini
Ketahui Cara Penularan Mpox dan Pencegahannya dengan Tepat
15 Negara Terbanyak Konsumsi Mi Instan di Dunia, Indonesia Nomor Berapa?
Gawat! Kimchi Terancam 'Lenyap' di Korsel Imbas Krisis Iklim
Leher-Pundak Kaku Pertanda Kolesterol Tinggi? Dokter Gizi Bilang Gini
Pemerintah Jepang Dikritik gegara Minta Wanita Pindah ke Desa usai Nikah
10 Manfaat Rebusan Sereh untuk Kesehatan, Amankah Diminum Tiap Hari?
Parahnya Kondisi Mpox di DR Kongo, Pasien Numpuk di RS Tapi Tak Bisa Diobati


In [ ]:
df=pd.read_csv("Crawl-berita.csv")
df

,judul,isi,tanggal,kategori
0,Apakah Obat Statin Cepat Turunkan Kadar Kolest...,Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\tCara Menur...,"Rabu, 04 Sep 2024 08:30 WIB",Kesehatan
1,Sesal IDI soal Viral Dokter Dilarang Pakai Hij...,Jakarta - Laporan dugaan aksi SARA di RS Medis...,"Rabu, 04 Sep 2024 08:00 WIB",Kesehatan
2,"5 Cara Menurunkan Kolesterol Secara Alami, Sal...",Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\tCara Menur...,"Rabu, 04 Sep 2024 07:00 WIB",Kesehatan
3,Ketahui Cara Penularan Mpox dan Pencegahannya ...,Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\tApa Itu Mp...,"Rabu, 04 Sep 2024 06:00 WIB",Kesehatan
4,15 Negara Terbanyak Konsumsi Mi Instan di Duni...,Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\tBerapa Bat...,"Rabu, 04 Sep 2024 05:00 WIB",Kesehatan
5,Gawat! Kimchi Terancam 'Lenyap' di Korsel Imba...,"Jakarta -\n \n Kimchi, makanan p...","Selasa, 03 Sep 2024 21:12 WIB",Kesehatan
6,Leher-Pundak Kaku Pertanda Kolesterol Tinggi? ...,Jakarta - Kolesterol tinggi dapat berbahaya un...,"Selasa, 03 Sep 2024 19:00 WIB",Kesehatan
7,Pemerintah Jepang Dikritik gegara Minta Wanita...,Jakarta - Pemerintah Jepang sebelumnya meminta...,"Selasa, 03 Sep 2024 18:00 WIB",Kesehatan
8,"10 Manfaat Rebusan Sereh untuk Kesehatan, Aman...",Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\tManfaat Re...,"Selasa, 03 Sep 2024 16:30 WIB",Kesehatan
9,"Parahnya Kondisi Mpox di DR Kongo, Pasien Nump...",Jakarta - Para pasien Mpox yang ada RD Kongo w...,"Selasa, 03 Sep 2024 16:01 WIB",Kesehatan
